In [5]:
!nvidia-smi

Sun Sep 29 06:38:50 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM3-32GB           Off |   00000000:05:00.0 Off |                    0 |
| N/A   28C    P0             61W /  350W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-0ihzu99o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-0ihzu99o
  Resolved https://github.com/huggingface/transformers.git to commit 2e24ee4dfa39cc0bc264b89edbccc373c8337086
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
from transformers import LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor,AutoTokenizer
import torch

In [8]:
from matplotlib import pyplot as plt
from matplotlib import animation
from io import BytesIO
import base64
import pandas as pd
import av
import numpy as np
import os

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")

model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    "llava-hf/LLaVA-NeXT-Video-7B-hf",
)

model.to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlavaNextVideoForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activati

In [10]:
data=pd.read_csv('dataset/baseline/train_data_categories.csv')
data.head()                 

,video_id,title,description,tags
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма..."
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура"
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира


In [11]:
data.shape

(1049, 4)

In [12]:
import numpy as np
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.

    Args:
        container (av.container.input.InputContainer): PyAV container.
        indices (List[int]): List of frame indices to decode.

    Returns:
        np.ndarray: np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])    

In [13]:
from huggingface_hub import hf_hub_download

# Download video from the hub
video_path_1 = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")

container = av.open(video_path_1)

# sample uniformly 8 frames from the video (we can sample more for longer videos)
total_frames = container.streams.video[0].frames
indices = np.arange(0, total_frames, total_frames / 8).astype(int)
clip_baby = read_video_pyav(container, indices)

In [14]:
iab_tags = pd.read_csv('dataset/baseline/IAB_tags.csv')
# Формируем список категорий с учетом иерархической структуры
categories = []
for i, row in iab_tags.iterrows():
    # Уровень 1 (родительская категория)
    if not pd.isna(row['Уровень 1 (iab)']):
        categories.append(row['Уровень 1 (iab)'])
    # Уровень 2 (подкатегория Уровня 1)
    if not pd.isna(row['Уровень 2 (iab)']):
        categories.append('{} > {}'.format(row['Уровень 1 (iab)'], row['Уровень 2 (iab)']))
    # Уровень 3 (подкатегория Уровня 2)
    if not pd.isna(row['Уровень 3 (iab)']):
        categories.append('{} > {} > {}'.format(row['Уровень 1 (iab)'], row['Уровень 2 (iab)'], row['Уровень 3 (iab)']))
categories_str = ', '.join(categories)        

In [15]:
len(categories_str)

44559

In [16]:
# Each "content" is a list of dicts and you can add image/video/text modalities
video_title = input("Введите название видео: ")
video_description = input("Введите описание видео:  ")

import random


# Преобразовать строку в список
categories_list = categories_str.split(', ')

# Сделать shuffle списка
random.shuffle(categories_list)

# Преобразовать список обратно в строку
categories_str = ', '.join(categories_list)
conversation = [
      {
          "role": "user",
          "content": [
              {"type": "text", "text": f"К какой категории относится данное видео? из списка:{categories_str[2048:4096]}"},
              {"type": "video", "title": video_title, "description": video_description},
              ],
      },
]
examples = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Определите, к какой из следующих категорий относится это видео: Транспорт, Спорт, Технологии. Ответьте одним или несколькими словами. Если видео не относится ни к одной из категорий, укажите предполагаемую категорию. Категории и их подгруппы должны браться только из предоставленного списка."},
            {"type": "video", "title": "Обзор нового автомобиля", "description": "Обзор нового автомобиля от компании XYZ."},
        ],
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "Транспорт > Типы кузова автомобиля > Седан"},
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Определите, к какой из следующих категорий относится это видео: Транспорт, Спорт, Технологии. Ответьте одним или несколькими словами. Если видео не относится ни к одной из категорий, укажите предполагаемую категорию. Категории и их подгруппы должны браться только из предоставленного списка."},
            {"type": "video", "title": "Как играть в футбол", "description": "Уроки по игре в футбол для начинающих."},
        ],
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "Спорт > Футбол"},
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Определите, к какой из следующих категорий относится это видео: Транспорт, Спорт, Технологии. Ответьте одним или несколькими словами. Если видео не относится ни к одной из категорий, укажите предполагаемую категорию. Категории и их подгруппы должны браться только из предоставленного списка."},
            {"type": "video", "title": "Поверь мне, подруга I #10 I Как стать популярной в школе", "description": "Поговорим о том, как вести себя в новой школе."},
        ],
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "Хобби и интересы"},
        ],
    },
]

Введите название видео:  Ребенок
Введите описание видео:   Ребенок читает книгу на кровати


In [17]:
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

In [18]:
prompt

'USER: <video>\nК какой категории относится данное видео? из списка:цина > Медицинские направления, Массовая культура, Спорт > Велоспорт, Бизнес и финансы, Фильмы и анимация, Религия и духовность, Карьера, Фильмы и анимация > Семейные и детские фильмы, Личные финансы > Инвестиции, Здоровый образ жизни > Методики оздоровления > Физиотерапия, Бизнес и финансы, Еда и напитки > Десерты и выпечка, Медицина > Медицинские направления > Диабет, Спорт > Баскетбол, Транспорт > Типы автомобилей > Подержанные автомобили, Музыка и аудио, Личные финансы > Инвестиции, Образование, Семья и отношения > Воспитание детей > Воспитание детей с особыми потребностями, Семья и отношения > Воспитание детей > Воспитание малышей, События и достопримечательности > Театральные мероприятия, Религия и духовность > Ислам, Спорт > Автогонки, Информационные технологии > Робототехника, Книги и литература > Кулинарные книги, События и достопримечательности, Карьера > Поиск работы > Ярмарки вакансий, Недвижимость > Промыш

In [19]:
# we still need to call the processor to tokenize the prompt and get pixel_values for videos
inputs = processor([prompt], videos=[clip_baby], padding=True, return_tensors="pt").to(model.device)

Expanding inputs for image/video tokens in LLaVa-NeXT-Video should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.


In [20]:
generate_kwargs = {"max_new_tokens": 50, "do_sample": True, "top_p": 0.95}

output = model.generate(**inputs, **generate_kwargs)
generated_text = processor.batch_decode(output, skip_special_tokens=True)

Expanding inputs for image.video tokens in LLaVa-NeXT-Video should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [35]:
print(generated_text)

['USER: \nК какой категории относится данное видео? из списка:цина > Медицинские направления, Массовая культура, Спорт > Велоспорт, Бизнес и финансы, Фильмы и анимация, Религия и духовность, Карьера, Фильмы и анимация > Семейные и детские фильмы, Личные финансы > Инвестиции, Здоровый образ жизни > Методики оздоровления > Физиотерапия, Бизнес и финансы, Еда и напитки > Десерты и выпечка, Медицина > Медицинские направления > Диабет, Спорт > Баскетбол, Транспорт > Типы автомобилей > Подержанные автомобили, Музыка и аудио, Личные финансы > Инвестиции, Образование, Семья и отношения > Воспитание детей > Воспитание детей с особыми потребностями, Семья и отношения > Воспитание детей > Воспитание малышей, События и достопримечательности > Театральные мероприятия, Религия и духовность > Ислам, Спорт > Автогонки, Информационные технологии > Робототехника, Книги и литература > Кулинарные книги, События и достопримечательности, Карьера > Поиск работы > Ярмарки вакансий, Недвижимость > Промышленная

In [36]:
for text in generated_text:
    if "ASSISTANT:" in text:
        print(text.split("ASSISTANT:")[1].strip())

Это видео принадлежит категории "Семья и отношения > Воспитание детей > Воспитание малышей".
